All groups and individual must do the following:

    V Convert the app sizes to a number
    V Convert the number of installs to a number
    V Transform “Varies with device” into a missing value
    V Convert Current Ver and Android Ver into a dotted number (e.g. 4.0.3 or 4.2)
    V Remove the duplicates
    For each category, compute the number of apps
    For each category, compute the average rating
    Create two dataframes: one for the genres and one bridging apps and genres. So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, once for Art & Design, once for Creativity
    For each genre, create a new column of the original dataframe. The new columns must have boolean values (True if the app has a given genre)
    For each genre, compute the average rating. What is the genre with highest average?
    For each app, compute the approximate income, obtain as a product of number of installs and price.
    For each app, compute its minimum and maximum Sentiment_polarity

The following part of the exercise must be done only by groups with two or three people

    For each app, compute the average number of words in its reviews
    For each app, compute its longest review
    For each app, compute the ratio between the number of installs and the number of reviews
    Cluster the apps according to the major android version (the first two digits — e.g. for 4.0.3 the major version is 4.0)
    For each cluster, compute the average date and the last date of an update.
    Excluding the free apps, what is the content rating with highest average price?

The following part of the exercise must be done only by groups with three people

    What is the genre with the highest total income?
    What is the genre with the highest fraction of free apps (over the number of all apps)?
    For each rating, compute the average income
    For each (Content Rating, Genre) pair, compute the number of reviews and the average rating.


In [1]:
'''
 Librerie utilizzate all'interno del progetto
'''

import pandas as pd
import numpy as np
import re
from datetime import datetime
from dateutil.parser import parse
import os

import ipywidgets as widgets #libreria per i widgets
from ipywidgets import interact, interactive, fixed, interact_manual

import yaml
from bokeh.io import output_notebook,curdoc
from bokeh.plotting import show
from bokeh.layouts import * #widgetbox
from bokeh.models import CustomJS, ColumnDataSource
from bokeh.models.widgets import Select, TextInput
output_notebook()

Loading BokehJS ...

## Definizione delle funzioni

In [2]:
'''
Conversione dell'attributo "Size" in un numero
    
    - Definizione dell'espressione regolare utilizzata per fare il parsing del valore
    - Creazione di un dizionario avente come chiavi i suffissi e come valore il moltiplicatore associato
    - Definizione della funzione per la conversione
'''

#string_to_num_size = re.compile('(?P<val>\d+\.{0,1}\d*)(?P<unit>[\w{0,1}|\+{0,1}])') #eliminando riga 10472 non ho più problema +
string_to_num_size = re.compile('(?P<val>\d+\.{0,1}\d*)(?P<unit>[\w{0,1}])')

unit_to_val = {'M' : 1000000,
               'k' : 1000,
               #'+' : 1000, #non è più necessario
               'Varies with device': np.nan}

def size_to_num(value):
    pippo = string_to_num_size.search(value.replace(",", ".")) #replace serve per sistemare formattazione errata di alcuni valori
    if(pippo != None):
        num = float(pippo.group('val'))
        return int(num)*unit_to_val[pippo.group('unit')]
    else:   
        return unit_to_val[value]

In [3]:
'''
Conversione dell'attributo "Installs" in un numero
    
    - Definizione dell'espressione regolare utilizzata per fare il parsing del valore
    - Definizione della funzione per la conversione
'''

string_to_num_installs = re.compile('(?P<val>\d+)(?P<plus>\+{0,1})')

def installs_to_num(value):
    pluto = string_to_num_installs.search(value.replace(",","")) #replace serve per sistemare formattazione errata di alcuni valori
    if(pluto != None):
        return int(pluto.group('val'))
    else:
        return np.nan

In [4]:
'''
Conversione degli attributi "Current Ver" e "Android Ver" in dotted number
    
    - Definizione dell'espressione regolare utilizzata per fare il parsing del valore
    - Definizione della funzione per la conversione
'''


'''
    Current Ver: Alcuni hanno formattazione a cazzo, con lettere o testo o più versioni
        - Troppe varianti per creare un'espressione regolare che matchi sempre
        - Cambio approccio, pattern che matcha dotted notation e la estrae dal campo
        - Se la trova la restituisce, tutto ciò che non è scritto come dotted_notation lo converto a 1.0
        
    Android Ver: Stesso approccio di Current Ver, trovo pattern dottet notation nel campo e lo restituisco,
                 Se non c'è restituisco 1.0, se ho più versioni restituisco l'ultima,
'''

dotted_notation = re.compile('(?P<num>\d+(\.\d+)*)')

def clean_current_ver(value):
    if(pd.isnull(value)):
        return '1.0'
    else:
        donald = dotted_notation.search(value)
        if(donald != None):
            return donald.group('num')
        else:
            return '1.0'
        
def clean_android_ver(value):
    daisy = dotted_notation.findall(value)
    if(daisy != None):
        return daisy[-1][0]
    else:
        return '1.0'

In [5]:
'''
Rimozione dei duplicati e funzioni di supporto

    La funzione prende in ingresso il dataframe da controllare e ne restituisce uno nuovo senza duplicati
    Per ogni nome di app estra un sottoinsieme del dataframe di partenza.
    Se il numero di righe è pari a 1 la aggiunge al nuovo dataframe, se è diverso viene effettuato
    un primo controllo utilizzando un metodo SQL-like tramite GroupBy.
    Se due righe hanno lo stesso valore per Current Ver, Android Ver e Last Updated mantendo solamente quella
    con il numero di reviews più alto (tramite la funzione lambda).
    Controllo il numero di righe che ha il sotto-dataframe che si è venuto a creare, se è pari a 1 ho eliminato
    i duplicati e aggiungo la riga al nuovo dataframe altrimenti effettuo un ulteriore controllo tramite la
    funzione check_update_version.
    Questa funzione ha il compito di controllare se le righe del sotto-dataframe rimanente si riferiscano ad
    app diverse, ma con stesso nome, oppure a versioni differenti della stessa app. Alla funzione viene passato
    il sotto-dataframe ordinato in maniera crescente di "Last Updated".
    Per controllare se siano più versioni o app diverse sono state create delle funzioni di supporto.
    is_same_genre serve per controllare che le app appartengano allo stesso genere.
    is_previous_update serve per controllare che l'attributo "last updated" dell'ultima riga sia il più recente
    is_next_version serve per controllare che la "Current Ver" dell'ultima riga sia più aggiornata rispetto alle altre righe
    has_more_reviews serve per controllare che l'ultima riga abbia un numero di reviews più alto delle altre
    Se queste condizioni sono soddisfatte allora mantendo solamente la riga corrispondente alla versione più
    aggiornata, altrimenti le app sono effettivamente diverse e mantengo tutte le righe nel nuovo dataframe.
    
'''

def remove_duplicate_rows(old_df):
    new_df = pd.DataFrame(columns=old_df.columns)
    app_name = old_df['App'].unique()
    for app in app_name:    
        tmp_df = old_df[old_df['App'] == app] 
        if(tmp_df.shape[0] != 1):            
            tmp_df = tmp_df.groupby(['Current Ver', 'Android Ver', 'Last Updated'], group_keys=False).apply(lambda x: x.loc[x['Reviews'].idxmax()]).reset_index(drop=True)           
            if(tmp_df.shape[0] != 1): 
                new_df.append(check_update_version(tmp_df.sort_values(by='Last Updated'))) 
            else:
                new_df = new_df.append(tmp_df)
        else:
            new_df = new_df.append(tmp_df)
    return new_df.reset_index(drop=True)


def check_update_version(to_check):
    if(len(to_check['Type'].unique()) == 1 and
      is_same_genre(to_check['Genres'].tolist()) and
      is_previous_update(to_check['Last Updated'].tolist()) and
      is_next_version(to_check['Current Ver'].tolist()) and
      has_more_reviews(to_check['Reviews'].tolist())): #allora è la versione aggiornata
        return to_check.iloc[-1:,]
    else:
        return to_check

def is_same_genre(genres):
    if(min(genres, key=len) in max(genres, key=len)): 
        return True
    else:
        return False
        
def is_previous_update(dates):
    if(max(dates) == dates[-1]): 
        return True
    else:
        return False
    

def is_next_version(versions): 
    non_digit = re.compile(r'[^\d]')
    new_versions = [re.sub(non_digit, '', str(x)) for x in versions]
    if(max(new_versions) == new_versions[-1]):
        return True
    else:
        return False
    
def has_more_reviews(reviews): 
    if(max(reviews) == reviews[-1]):
        return True
    else:
        return False

In [6]:
'''
 Caricamento del percorso dei dataset
'''

gps_path = os.getcwd() + "\\googleplaystore.csv"
gps_ur_path = os.getcwd() + "\\googleplaystore_user_reviews.csv"

In [7]:
gps = pd.read_csv(gps_path, low_memory = False)
gps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [8]:
'''
Operazioni preliminari sul Dataset googleplaystore

    - Eliminazione della riga numero 10472 in quanto i campi sono formattati in modo errato
    - Eliminazione delle righe in cui i campi "Current Ver" e "Android Ver" non sono presenti
      o sono uguali a "Varies with device"
    - Eliminazione dei duplicati attraverso la funzione drop_duplicated() di pandas, elimina 
      solamente le tuple con tutti gli attributi uguali
    - Conversione dell'attributo "Reviews" a numeric
    - Conversione dell'attributo "Last Updated" a datetime
    - Conversione dell'attributo "Rating" a numeric

      
'''

gps = gps.drop(gps.index[10472])
gps = gps.drop_duplicates()
gps = gps[gps['Android Ver'].notnull()]
gps = gps[gps['Android Ver'] != 'Varies with device']
gps = gps[gps['Current Ver'] != 'Varies with device']


gps['Reviews'] = pd.to_numeric(gps['Reviews'], downcast = 'float')
gps['Last Updated'] = pd.to_datetime(gps['Last Updated'])
gps['Rating'] = pd.to_numeric(gps['Rating'], downcast='float')


In [9]:
'''
    - Applicazione delle funzioni definite precedentemente per i punti da 1 a 5
    - Eliminazione delle colonne non più necessarie
    - Rename delle colonne sistemate
    - Cast degli attributo al tipo corretto
    
'''

gps['IntSize'] = gps['Size'].apply(size_to_num)
gps['IntInstalls'] = gps['Installs'].apply(installs_to_num)
gps['Android Ver ok'] = gps['Android Ver'].apply(clean_android_ver)
gps['Current Ver ok'] = gps['Current Ver'].apply(clean_current_ver)

cleaned_gps = remove_duplicate_rows(gps)

to_drop = ['Size', 'Installs', 'Android Ver', 'Current Ver']
cleaned_gps.drop(to_drop, axis = 1, inplace=True)

new_columns = ['App', 'Category','Rating','Reviews','Type','Price','Content Rating','Genres','Last Updated',
               'Size', 'Installs', 'Android Ver', 'Current Ver']
cleaned_gps.columns = new_columns

cleaned_gps['Installs'] = pd.to_numeric(cleaned_gps['Installs'], downcast='integer')
cleaned_gps['Size'] = pd.to_numeric(cleaned_gps['Size'], downcast='integer')
cleaned_gps['Reviews'] = pd.to_numeric(cleaned_gps['Reviews'], downcast='integer')

In [10]:
cleaned_gps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8524 entries, 0 to 8523
Data columns (total 13 columns):
App               8524 non-null object
Category          8524 non-null object
Rating            7108 non-null float64
Reviews           8524 non-null int32
Type              8524 non-null object
Price             8524 non-null object
Content Rating    8524 non-null object
Genres            8524 non-null object
Last Updated      8524 non-null datetime64[ns]
Size              8340 non-null float64
Installs          8524 non-null int32
Android Ver       8524 non-null object
Current Ver       8524 non-null object
dtypes: datetime64[ns](1), float64(2), int32(2), object(8)
memory usage: 799.2+ KB


V For each category, compute the number of apps
V For each category, compute the average rating
V Create two dataframes: one for the genres and one bridging apps and genres. So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, once for Art & Design, once for Creativity

 ? For each genre, create a new column of the original dataframe. The new columns must have boolean values (True if the app has a given genre)

V For each genre, compute the average rating. What is the genre with highest average?
V For each app, compute the approximate income, obtain as a product of number of installs and price.
For each app, compute its minimum and maximum Sentiment_polarity

In [11]:
'''
    Numero di app e rating medio per ogni categoria
'''
#calcolo numero di app per categoria
category_name = cleaned_gps['Category'].unique()
category_num = []
category_rating = []
for cat in category_name:
    category_num.append(len(cleaned_gps[cleaned_gps['Category'] == cat]))
    category_rating.append(round(cleaned_gps[cleaned_gps['Category'] == cat]['Rating'].mean(),2))
cat_dict = {'Category': category_name, 'n_app': category_num, 'avg_rating': category_rating}
cat_df = pd.DataFrame(cat_dict)
cat_df

,Category,avg_rating,n_app
0,ART_AND_DESIGN,4.36,58
1,FAMILY,4.18,1776
2,AUTO_AND_VEHICLES,4.16,76
3,BEAUTY,4.30,48
4,BOOKS_AND_REFERENCE,4.33,196
5,BUSINESS,4.10,381
6,COMICS,4.20,49
7,COMMUNICATION,4.09,251
8,DATING,3.97,164
9,EDUCATION,4.37,80


In [12]:
#df generi
genre_list = [] #df  generi

#df app e generi
app_list = [] #colonna app ripetute 
genre_app_list = []
rating_list = []
avg_rating_gen = []
avg_income_app = []

price = re.compile('(?P<num>\d+\.{0,1}\d*)')
semicolumn = re.compile(';')

for index, row in cleaned_gps.iterrows(): 
    gn = semicolumn.search(row['Genres'])
    appName = row['App']
    price = row['Price'] 
    if ( gn != None):
        listgen = row['Genres'].split(';')
        for g in listgen:
            if (g not in genre_list):
                genre_list.append(g)
            app_list.append(appName)
            genre_app_list.append(g)
            rating_list.append(row['Rating'])            
    else:
        if(row['Genres'] not in genre_list):
            genre_list.append(row['Genres'])
        app_list.append(appName)
        genre_app_list.append(row['Genres'])
        rating_list.append(row['Rating'])
   


app_genre_df = pd.DataFrame({'App' : app_list, 'Genres' : genre_app_list, 'Rating' : rating_list})

for gen in app_genre_df['Genres'].unique():
    avg_rating_gen.append(round(app_genre_df[app_genre_df['Genres'] == gen]['Rating'].mean(),2))

genre_df = pd.DataFrame({'Genres' : genre_list, 'Avg Rating' : avg_rating_gen})
#genre_rating_df = pd.DataFrame({'Genres' : genre_list, 'Rating' : app_rating})

In [13]:
genre_df.sort_values('Avg Rating', ascending = False)

,Avg Rating,Genres
15,4.49,Events
27,4.37,Puzzle
0,4.36,Art & Design
12,4.35,Brain Games
2,4.34,Creativity
30,4.34,Word
45,4.32,Personalization
6,4.32,Books & Reference
47,4.30,Parenting
5,4.30,Beauty


In [14]:
price = re.compile('(?P<num>\d+\.{0,1}\d*)')
def return_income(row):
    p = price.search(str(row['Price']))
    if(p != None):
        prezzo = p.group('num')
        income = float(prezzo)*row['Installs']
        return income
    else:
        return '0' 

In [15]:
prova = cleaned_gps
prova['Income'] = prova.apply(return_income, axis = 1)
prova[prova['Type'] != 'Free']

,App,Category,Rating,Reviews,Type,Price,Content Rating,Genres,Last Updated,Size,Installs,Android Ver,Current Ver,Income
188,TurboScan: scan documents and receipts in PDF,BUSINESS,4.7,11442,Paid,$4.99,Everyone,Business,2018-03-25,6000000.0,100000,4.0,1.5.2,499000.00
189,Tiny Scanner Pro: PDF Doc Scan,BUSINESS,4.8,10295,Paid,$4.99,Everyone,Business,2017-04-11,39000000.0,100000,3.0,3.4.6,499000.00
288,Puffin Browser Pro,COMMUNICATION,4.0,18247,Paid,$3.99,Everyone,Communication,2018-07-05,NaN,100000,4.1,7.5.3.20547,399000.00
313,"Moco+ - Chat, Meet People",DATING,4.2,1546,Paid,$3.99,Mature 17+,Dating,2018-06-19,NaN,10000,4.1,2.6.139,39900.00
314,Calculator,DATING,2.6,57,Paid,$6.99,Everyone,Dating,2017-10-25,6000000.0,1000,4.0,1.1.6,6990.00
315,Truth or Dare Pro,DATING,NaN,0,Paid,$1.49,Teen,Dating,2017-09-01,20000000.0,50,4.0,1.0,74.50
316,"Private Dating, Hide App- Blue for PrivacyHider",DATING,NaN,0,Paid,$2.99,Everyone,Dating,2017-07-25,18000.0,100,4.0,1.0.1,299.00
317,Ad Blocker for SayHi,DATING,NaN,4,Paid,$3.99,Teen,Dating,2018-08-02,1000000.0,100,4.0.3,1.2,399.00
318,AMBW Dating App: Asian Men Black Women Interra...,DATING,3.5,2,Paid,$7.99,Mature 17+,Dating,2017-01-21,17000000.0,100,4.0,1.0.1,799.00
556,Sago Mini Hat Maker,EDUCATION,4.9,11,Paid,$3.99,Everyone,Education;Pretend Play,2017-07-24,63000000.0,1000,4.0.3,1.0,3990.00


In [16]:
#calcolo dell'average rating per categoria
# Set up widget
select = Select(title="Selezionare categoria:", value=cleaned_gps['Category'].unique()[0], options=list(cleaned_gps['Category'].unique()))
text_1 = TextInput(title="Numero di App nella categoria:", value = '')
text_2 = TextInput(title="Rating medio:", value = '')

source = ColumnDataSource(data=cat_df)
# Set up callbacks

callback = CustomJS(args=dict(source = source, text_1 = text_1, text_2 = text_2), code="""
    text_1.value = cb_obj.value;
    text_2.value = source.data['Category' == cb_obj.value];
    
""")

select.js_on_change('value', callback)
inputs = widgetbox(select, text_1, text_2)

show(inputs)

In [55]:
gps_ur = pd.read_csv('googleplaystore_user_reviews.csv', low_memory = False)
#gps_ur.info()
gps_ur[gps_ur['Sentiment'] == 'Positive'].min()

App                                                   10 Best Foods for You
Translated_Review         "Classic" allows untimed play; game holds posi...
Sentiment                                                          Positive
Sentiment_Polarity                                              5.55112e-18
Sentiment_Subjectivity                                                    0
dtype: object

In [66]:
def get_length_review(text):
    if ( pd.isnull(text)):
        return 0
    else:
        return len(text.split()) 

In [73]:
gps_ur['LengthRew'] = gps_ur['Translated_Review'].apply(get_length_review)
gps_ur['LengthRew'] = pd.to_numeric(gps_ur['LengthRew'], downcast = 'integer')

avg_text = []
longest_rew = []
longest_text = []
for app in gps_ur['App'].unique():
    avg_text.append(round(gps_ur[gps_ur['App']==app]['LengthRew'].mean(),2))
    longest_rew.append(gps_ur[gps_ur['App']==app]['LengthRew'].max())
    longest_text.append(gps_ur[(gps_ur['App']==app)& (gps_ur['LengthRew'].max())]['Translated_Review'].iloc[0,])
app_text_len = pd.DataFrame({'App': gps_ur['App'].unique(), 'avg_rev': avg_text, 'maxLen': longest_rew , 'maxText': longest_text}).sort_values('avg_rev', ascending = False)
app_text_len

,App,avg_rev,maxLen,maxText
513,Crunchyroll - Everything Anime,44.72,64,Great app. Works like charm me. Main complaint...
1001,Hacker's Keyboard,44.12,105,"Some damn fine work. I'm novelist, far best ke..."
1019,Helix Jump,40.67,102,"Actually really enjoy playing game, however I ..."
223,BeWild Free Dating & Chat App,40.15,82,Wow amazing dating app. The registration proce...
447,Clash Royale,38.63,70,The game favors big spenders rigged deck rotat...
598,Down Dog: Great Yoga Anywhere,38.38,76,One best I could find yoga home. I yoga expert...
120,"Android Auto - Maps, Media, Messaging & Voice",38.25,78,I every drive. In car I secure phone dash run ...
384,Candy Crush Saga,38.25,101,"If get free lives refill, continue accumulate ..."
1059,Honkai Impact 3rd,37.92,112,It's super fun I absolutely love graphics! The...
435,Choices: Stories You Play,37.02,87,Overall game good could great put diamonds nee...


In [63]:
text

"0        I like eat delicious food. That's I'm cooking ...\n1          This help eating healthy exercise regular basis\n2                                                      NaN\n3               Works great especially going grocery store\n4                                             Best idea us\n5                                                 Best way\n6                                                  Amazing\n7                                                      NaN\n8                                     Looking forward app,\n9                    It helpful site ! It help foods get !\n10                                               good you.\n11       Useful information The amount spelling errors ...\n12       Thank you! Great app!! Add arthritis, eyes, im...\n13       Greatest ever Completely awesome maintain heal...\n14       Good health...... Good health first priority.....\n15                                                     NaN\n16       Health It's important world ei